In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import cv2

import time
from collections import namedtuple
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output

In [2]:
Label = namedtuple( 'Label' , [

    'name'        , # The identifier of this label, e.g. 'car', 'person', ... .
                    # We use them to uniquely name a class

    'id'          , # An integer ID that is associated with this label.
                    # The IDs are used to represent the label in ground truth images
                    # An ID of -1 means that this label does not have an ID and thus
                    # is ignored when creating ground truth images (e.g. license plate).
                    # Do not modify these IDs, since exactly these IDs are expected by the
                    # evaluation server.

    'trainId'     , # Feel free to modify these IDs as suitable for your method. Then create
                    # ground truth images with train IDs, using the tools provided in the
                    # 'preparation' folder. However, make sure to validate or submit results
                    # to our evaluation server using the regular IDs above!
                    # For trainIds, multiple labels might have the same ID. Then, these labels
                    # are mapped to the same class in the ground truth images. For the inverse
                    # mapping, we use the label that is defined first in the list below.
                    # For example, mapping all void-type classes to the same ID in training,
                    # might make sense for some approaches.
                    # Max value is 255!

    'category'    , # The name of the category that this label belongs to

    'categoryId'  , # The ID of this category. Used to create ground truth images
                    # on category level.

    'hasInstances', # Whether this label distinguishes between single instances or not

    'ignoreInEval', # Whether pixels having this class as ground truth label are ignored
                    # during evaluations or not

    'color'       , # The color of this label
    ] )


#--------------------------------------------------------------------------------
# A list of all labels
#--------------------------------------------------------------------------------

# Please adapt the train IDs as appropriate for your approach.
# Note that you might want to ignore labels with ID 255 during training.
# Further note that the current train IDs are only a suggestion. You can use whatever you like.
# Make sure to provide your results using the original IDs and not the training IDs.
# Note that many IDs are ignored in evaluation and thus you never need to predict these!

labels = [
    #       name                     id    trainId   category            catId     hasInstances   ignoreInEval   color
    Label(  'unlabeled'            ,  0 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'ego vehicle'          ,  1 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'rectification border' ,  2 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'out of roi'           ,  3 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'static'               ,  4 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'dynamic'              ,  5 ,      255 , 'void'            , 0       , False        , True         , (111, 74,  0) ),
    Label(  'ground'               ,  6 ,      255 , 'void'            , 0       , False        , True         , ( 81,  0, 81) ),
    Label(  'road'                 ,  7 ,        0 , 'flat'            , 1       , False        , False        , (128, 64,128) ),
    Label(  'sidewalk'             ,  8 ,        1 , 'flat'            , 1       , False        , False        , (244, 35,232) ),
    Label(  'parking'              ,  9 ,      255 , 'flat'            , 1       , False        , True         , (250,170,160) ),
    Label(  'rail track'           , 10 ,      255 , 'flat'            , 1       , False        , True         , (230,150,140) ),
    Label(  'building'             , 11 ,        2 , 'construction'    , 2       , False        , False        , ( 70, 70, 70) ),
    Label(  'wall'                 , 12 ,        3 , 'construction'    , 2       , False        , False        , (102,102,156) ),
    Label(  'fence'                , 13 ,        4 , 'construction'    , 2       , False        , False        , (190,153,153) ),
    Label(  'guard rail'           , 14 ,      255 , 'construction'    , 2       , False        , True         , (180,165,180) ),
    Label(  'bridge'               , 15 ,      255 , 'construction'    , 2       , False        , True         , (150,100,100) ),
    Label(  'tunnel'               , 16 ,      255 , 'construction'    , 2       , False        , True         , (150,120, 90) ),
    Label(  'pole'                 , 17 ,        5 , 'object'          , 3       , False        , False        , (153,153,153) ),
    Label(  'polegroup'            , 18 ,      255 , 'object'          , 3       , False        , True         , (153,153,153) ),
    Label(  'traffic light'        , 19 ,        6 , 'object'          , 3       , False        , False        , (250,170, 30) ),
    Label(  'traffic sign'         , 20 ,        7 , 'object'          , 3       , False        , False        , (220,220,  0) ),
    Label(  'vegetation'           , 21 ,        8 , 'nature'          , 4       , False        , False        , (107,142, 35) ),
    Label(  'terrain'              , 22 ,        9 , 'nature'          , 4       , False        , False        , (152,251,152) ),
    Label(  'sky'                  , 23 ,       10 , 'sky'             , 5       , False        , False        , ( 70,130,180) ),
    Label(  'person'               , 24 ,       11 , 'human'           , 6       , True         , False        , (220, 20, 60) ),
    Label(  'rider'                , 25 ,       12 , 'human'           , 6       , True         , False        , (255,  0,  0) ),
    Label(  'car'                  , 26 ,       13 , 'vehicle'         , 7       , True         , False        , (  0,  0,142) ),
    Label(  'truck'                , 27 ,       14 , 'vehicle'         , 7       , True         , False        , (  0,  0, 70) ),
    Label(  'bus'                  , 28 ,       15 , 'vehicle'         , 7       , True         , False        , (  0, 60,100) ),
    Label(  'caravan'              , 29 ,      255 , 'vehicle'         , 7       , True         , True         , (  0,  0, 90) ),
    Label(  'trailer'              , 30 ,      255 , 'vehicle'         , 7       , True         , True         , (  0,  0,110) ),
    Label(  'train'                , 31 ,       16 , 'vehicle'         , 7       , True         , False        , (  0, 80,100) ),
    Label(  'motorcycle'           , 32 ,       17 , 'vehicle'         , 7       , True         , False        , (  0,  0,230) ),
    Label(  'bicycle'              , 33 ,       18 , 'vehicle'         , 7       , True         , False        , (119, 11, 32) ),
    Label(  'license plate'        , -1 ,       -1 , 'vehicle'         , 7       , False        , True         , (  0,  0,142) ),
]

In [3]:
name2label      = { label.name    : label for label in labels           }
# id to label object
id2label        = { label.id      : label for label in labels           }
# trainId to label object
trainId2label   = { label.trainId : label for label in reversed(labels) }

In [4]:
dataset_root = './Data/cityscapes'

In [5]:
# Define transformation, 
# ToTensor : Convert PIL image to Pytorch tensor
# Resize   : Resize image to from (1024, 2048) to (128, 256)
# Resize   : Use nearest-neighbour interpolation to keep the same values from original image 

transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize(size=(128, 256), interpolation=transforms.InterpolationMode.NEAREST)
            ])

In [6]:
# Define dataset
train_data = torchvision.datasets.Cityscapes(root=dataset_root, split="train", mode="fine", target_type="semantic", transform=transform, target_transform=transform)
valid_data = torchvision.datasets.Cityscapes(root=dataset_root, split="val", mode="fine", target_type="semantic", transform=transform, target_transform=transform)

# Only use 1000 training example to reduce training time 
train_data, _ = torch.utils.data.random_split(train_data, [1000, 1975], generator=torch.Generator().manual_seed(10))

# Split valid_data into valid_data and test_data because the cityscapes test dataset has problems
split = 0.5
n_valid_examples = int(len(valid_data)*split)
n_test_examples = len(valid_data) - n_valid_examples

valid_data, test_data = torch.utils.data.random_split(valid_data, [n_valid_examples, n_test_examples], generator=torch.Generator().manual_seed(10))

In [7]:
# Check length of all datasets
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 1000
Number of validation examples: 250
Number of testing examples: 250


In [8]:
# Define dataloader
batch_size = 50
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, batch_size=batch_size)
test_loader  = DataLoader(test_data, batch_size=batch_size)

In [13]:
# Change id to trainId for all labels
def changeLabel(dataset, loader, batch_size):
    storeData = np.zeros((len(dataset), 3, 128, 256))
    storeLabel = np.zeros((len(dataset), 128, 256))
    
    for batch_id, (data, label) in enumerate(loader):
        storeData[batch_id*batch_size:(batch_id+1)*batch_size] = data
    
        # Loop every image in a batch
        for i in range(batch_size):
            target = label[i].squeeze()
            size = target.numpy().shape
            target = target.reshape(target.numel()).numpy()
            target = (target*255).astype('int')
            target = np.where(target == 7, target*0, target)
            target = np.where(target == 8, target*0+1, target)
            target = np.where(target == 11, target*0+2, target)
            target = np.where(target == 12, target*0+3, target)
            target = np.where(target == 13, target*0+4, target)
            target = np.where(target == 17, target*0+5, target)
            target = np.where(target == 19, target*0+6, target)
            target = np.where(target == 20, target*0+7, target)
            target = np.where(target == 21, target*0+8, target)
            target = np.where(target == 22, target*0+9, target)
            target = np.where(target == 23, target*0+10, target)
            target = np.where(target == 24, target*0+11, target)
            target = np.where(target == 25, target*0+12, target)
            target = np.where(target == 26, target*0+13, target)
            target = np.where(target == 27, target*0+14, target)
            target = np.where(target == 28, target*0+15, target)
            target = np.where(target == 31, target*0+16, target)
            target = np.where(target == 32, target*0+17, target)
            target = np.where(target == 33, target*0+18, target)
            target = np.where(target > 18, target*0+19, target)
            target = target.reshape(size)
            storeLabel[i+(batch_id*batch_size)] = target
    
    return (storeData, storeLabel)

In [14]:
trainData, trainLabel = changeLabel(dataset=train_data, loader=train_loader, batch_size=batch_size)
validData, validLabel = changeLabel(dataset=valid_data, loader=valid_loader, batch_size=50)
testData, testLabel = changeLabel(dataset=test_data, loader=test_loader, batch_size=batch_size)

In [15]:
np.savez_compressed('cityScapes.npz', trainData, trainLabel, validData, validLabel, testData, testLabel)